In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class TestDataset(Dataset):
    def __init__(self, data_nums = 10000):
        self.dataset = []
        for i in range(data_nums):
            data = {}
            data["input"] = torch.randn(1,10)
#             data["output"] = torch.tensor([[sum(data["input"][0][max(0, i-1):min(i+2, 11)]) for i in range(len(data["input"][0]))]])
            data["output"] = torch.argsort(data["input"]).float()
#             data["output"] = torch.randn(1,10) + 10
            self.dataset.append(data)
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        return self.dataset[idx]

In [2]:
class Model(nn.Module):
    def __init__(self, input_length = 10, output_length = 10):
        super(Model, self).__init__()

        self.seq = nn.Sequential(
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Tanh(),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Tanh(),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Tanh(),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Tanh(),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
        )
#         self.Conv2d = nn.Conv2d(1,1,kernel_size=3,padding=1)
    def forward(self, data):
        out = self.seq(data)
        return out
        

In [3]:
dataset = TestDataset()
model = Model()

In [4]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(model.parameters(), lr=0.0001)
MSE = nn.MSELoss()
CS = nn.CosineSimilarity(dim = 2)

data_loader = torch.utils.data.DataLoader(dataset, batch_size = 4096, shuffle = True)
model.to(device)


Model(
  (seq): Sequential(
    (0): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (2): Tanh()
    (3): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): Tanh()
    (6): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (7): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (8): Tanh()
    (9): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (10): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (11): Tanh()
    (12): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (13): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
  )
)

In [ ]:
from tqdm import tqdm
import numpy as np
import copy

before_param_data = {}


for name, param in model.named_parameters():
    if param.requires_grad:
        before_param_data[name] = copy.deepcopy(param.data)
        
for epoch in range(1000000):
    loss_mean = []
    for i, batch in (enumerate(data_loader)):
        input_data, GT = (batch["input"].to(device), batch["output"].to(device))

        optimizer.zero_grad()
        predictTE = model(input_data)
#         sort_idx = torch.argsort(predictTE, dim = 2).float()
#         predictTE = predictTE * 0 + sort_idx


        loss = MSE(predictTE, GT)
        loss = loss + torch.mean(1. - (CS(predictTE, GT))) * 5
#         print(input_data.shape)
#         print(GT.shape)
#         assert False, "AA"
#         print(GT[0])
#         print(loss.shape)
#         print(torch.sum(loss))
        loss.backward()
        optimizer.step()
        
        loss_mean.append(loss.cpu().item())
    if epoch % 20 == 19:
        print(np.mean(loss_mean))
        for name, param in model.named_parameters():
            if param.requires_grad:
#                 print(name, param.data)
#                 print(name, before_param_data[name])
#                 print(predictTE[0])
#                 print(GT[0])

#                 print((before_param_data[name] == param.data).any())
#                 assert (before_param_data[name] != param.data).any(), "BACK PROPAGATION FAIL!!!!!!!!!!!!!!!!!"
#                 break
                pass
        print(predictTE[0])
        print(torch.argsort(predictTE[0]).float())
        print(GT[0])
#         print(loss.requires_grad)
        print("=" * 80)

42.969295501708984
tensor([[-0.5206, -0.5539, -0.5459, -0.5489, -0.5502, -0.5463, -0.5478, -0.5452,
         -0.5458, -0.5977]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 1., 4., 3., 6., 5., 2., 8., 7., 0.]], device='cuda:0')
tensor([[3., 2., 8., 0., 1., 7., 5., 4., 6., 9.]], device='cuda:0')
42.88004938761393
tensor([[-0.5165, -0.5519, -0.5386, -0.5346, -0.5356, -0.5380, -0.5401, -0.5358,
         -0.5394, -0.5825]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 1., 6., 8., 2., 5., 7., 4., 3., 0.]], device='cuda:0')
tensor([[9., 6., 0., 3., 5., 7., 2., 4., 1., 8.]], device='cuda:0')
42.79754130045573
tensor([[-0.5100, -0.5406, -0.5292, -0.5304, -0.5318, -0.5313, -0.5337, -0.5284,
         -0.5289, -0.5662]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 1., 6., 4., 5., 3., 2., 8., 7., 0.]], device='cuda:0')
tensor([[2., 4., 0., 3., 6., 1., 9., 7., 8., 5.]], device='cuda:0')
42.71972529093424
tensor([[-0.5058, -0.5393, -0.5227, -0.5190, -0.5192, -0.519

28.642398198445637
tensor([[ 0.0979,  0.2445,  0.3010,  0.3286,  0.3359,  0.3363,  0.3355,  0.3282,
          0.2464, -0.3787]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 0., 1., 8., 2., 7., 3., 6., 4., 5.]], device='cuda:0')
tensor([[7., 5., 9., 3., 4., 8., 2., 1., 6., 0.]], device='cuda:0')
28.0517152150472
tensor([[ 0.1324,  0.2880,  0.3498,  0.3793,  0.3864,  0.3866,  0.3858,  0.3786,
          0.2921, -0.3560]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 0., 1., 8., 2., 7., 3., 6., 4., 5.]], device='cuda:0')
tensor([[2., 9., 4., 8., 1., 5., 6., 3., 7., 0.]], device='cuda:0')
27.55445162455241
tensor([[ 0.1635,  0.3276,  0.3952,  0.4268,  0.4341,  0.4343,  0.4333,  0.4254,
          0.3339, -0.3344]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 0., 1., 8., 2., 7., 3., 6., 4., 5.]], device='cuda:0')
tensor([[1., 5., 3., 0., 6., 7., 2., 8., 4., 9.]], device='cuda:0')
27.11797523498535
tensor([[ 0.1954,  0.3677,  0.4400,  0.4729,  0.4798,  0.4799

20.300134658813477
tensor([[0.8387, 1.2070, 1.3897, 1.4376, 1.4423, 1.4423, 1.4415, 1.4349, 1.2448,
         0.1435]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 0., 1., 8., 2., 7., 3., 6., 5., 4.]], device='cuda:0')
tensor([[4., 1., 5., 2., 9., 8., 7., 0., 3., 6.]], device='cuda:0')
19.957704544067383
tensor([[0.8776, 1.2593, 1.4487, 1.4969, 1.5016, 1.5016, 1.5008, 1.4942, 1.2982,
         0.1728]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 0., 1., 8., 2., 7., 3., 6., 4., 5.]], device='cuda:0')
tensor([[5., 9., 0., 2., 8., 1., 4., 6., 7., 3.]], device='cuda:0')
19.60641924540202
tensor([[0.9172, 1.3125, 1.5086, 1.5570, 1.5615, 1.5615, 1.5607, 1.5544, 1.3526,
         0.2026]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 0., 1., 8., 2., 7., 3., 6., 4., 5.]], device='cuda:0')
tensor([[3., 9., 4., 8., 1., 5., 7., 6., 2., 0.]], device='cuda:0')
19.268842061360676
tensor([[0.9578, 1.3668, 1.5695, 1.6179, 1.6224, 1.6224, 1.6216, 1.6151, 1.4073,
       

13.697958946228027
tensor([[1.7701, 2.4796, 2.8106, 2.8537, 2.8560, 2.8560, 2.8555, 2.8509, 2.5258,
         0.8660]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 0., 1., 8., 2., 7., 3., 6., 5., 4.]], device='cuda:0')
tensor([[8., 1., 9., 4., 7., 0., 2., 5., 3., 6.]], device='cuda:0')
13.459889729817709
tensor([[1.8128, 2.5392, 2.8769, 2.9195, 2.9218, 2.9218, 2.9213, 2.9167, 2.5855,
         0.9005]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 0., 1., 8., 2., 7., 3., 6., 5., 4.]], device='cuda:0')
tensor([[7., 3., 0., 1., 9., 6., 4., 2., 5., 8.]], device='cuda:0')
13.254907608032227
tensor([[1.8556, 2.5989, 2.9430, 2.9851, 2.9873, 2.9873, 2.9868, 2.9823, 2.6450,
         0.9350]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 0., 1., 8., 2., 7., 3., 6., 5., 4.]], device='cuda:0')
tensor([[3., 2., 9., 8., 6., 0., 4., 7., 1., 5.]], device='cuda:0')
13.039346059163412
tensor([[1.8980, 2.6582, 3.0088, 3.0504, 3.0525, 3.0525, 3.0520, 3.0475, 2.7041,
      

10.583959261576334
tensor([[2.6074, 3.6629, 4.1198, 4.1526, 4.1538, 4.1537, 4.1534, 4.1502, 3.7076,
         1.5623]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 0., 1., 8., 2., 7., 3., 6., 5., 4.]], device='cuda:0')
tensor([[4., 7., 8., 6., 9., 3., 0., 2., 5., 1.]], device='cuda:0')
10.54574998219808
tensor([[2.6377, 3.7060, 4.1673, 4.1994, 4.2005, 4.2005, 4.2002, 4.1970, 3.7503,
         1.5882]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 0., 1., 8., 2., 7., 3., 6., 5., 4.]], device='cuda:0')
tensor([[0., 7., 5., 1., 2., 9., 4., 6., 3., 8.]], device='cuda:0')
10.46032969156901
tensor([[2.6666, 3.7475, 4.2130, 4.2447, 4.2458, 4.2458, 4.2455, 4.2423, 3.7919,
         1.6134]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 0., 1., 8., 2., 7., 3., 6., 5., 4.]], device='cuda:0')
tensor([[7., 2., 8., 3., 6., 9., 4., 0., 1., 5.]], device='cuda:0')
10.433463414510092
tensor([[2.6944, 3.7875, 4.2572, 4.2885, 4.2895, 4.2895, 4.2892, 4.2861, 3.8320,
        

...

8.343946774800619
tensor([[3.7475, 3.5910, 3.8590, 4.3705, 4.3297, 4.7756, 4.7972, 5.1757, 5.6600,
         5.9392]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[1., 0., 2., 4., 3., 5., 6., 7., 8., 9.]], device='cuda:0')
tensor([[0., 7., 1., 2., 4., 8., 3., 5., 9., 6.]], device='cuda:0')
8.336516698201498
tensor([[5.7963, 5.8074, 5.0391, 4.7636, 4.9764, 4.2177, 4.3039, 3.4126, 2.4898,
         2.5917]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[8., 9., 7., 5., 6., 3., 4., 2., 0., 1.]], device='cuda:0')
tensor([[9., 6., 8., 3., 4., 7., 1., 5., 0., 2.]], device='cuda:0')
8.333108266194662
tensor([[3.4823, 4.5771, 5.1079, 4.0050, 4.7596, 3.9601, 4.1460, 3.8083, 4.7165,
         5.1485]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[0., 7., 5., 3., 6., 1., 8., 4., 2., 9.]], device='cuda:0')
tensor([[9., 1., 6., 3., 7., 4., 5., 2., 0., 8.]], device='cuda:0')
8.341689427693685
tensor([[2.6362, 1.9470, 2.9811, 4.3333, 4.0813, 4.8859, 4.9495, 5.2455, 6.3493,
         6

8.332165082295736
tensor([[4.8317, 4.6089, 4.3532, 4.7945, 4.4696, 4.5919, 4.3087, 4.5195, 4.2876,
         4.1267]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 8., 6., 2., 4., 7., 5., 1., 3., 0.]], device='cuda:0')
tensor([[8., 6., 0., 2., 1., 5., 4., 9., 3., 7.]], device='cuda:0')
8.334583282470703
tensor([[2.6234, 2.8129, 3.6542, 4.4806, 4.0581, 4.8676, 4.6834, 5.1596, 5.6306,
         6.1076]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[0., 1., 2., 4., 3., 6., 5., 7., 8., 9.]], device='cuda:0')
tensor([[4., 2., 1., 5., 0., 6., 9., 8., 7., 3.]], device='cuda:0')
8.333053906758627
tensor([[2.3648, 2.5096, 3.6370, 4.4323, 4.2954, 4.7482, 4.6381, 4.8063, 5.7500,
         6.3242]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[0., 1., 2., 4., 3., 6., 5., 7., 8., 9.]], device='cuda:0')
tensor([[1., 6., 0., 5., 4., 2., 3., 9., 8., 7.]], device='cuda:0')
8.334211349487305
tensor([[5.9554, 5.7517, 5.0662, 4.7219, 4.6508, 4.0384, 4.4109, 3.7481, 3.6916,
         4

8.335071563720703
tensor([[6.2050, 5.8731, 4.9707, 4.8525, 4.9059, 4.2764, 4.1297, 3.8319, 2.8628,
         2.2997]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 8., 7., 6., 5., 3., 4., 2., 1., 0.]], device='cuda:0')
tensor([[5., 8., 3., 7., 0., 9., 1., 6., 4., 2.]], device='cuda:0')
8.342089653015137
tensor([[4.6660, 4.4201, 4.3413, 4.2910, 4.3032, 4.4861, 4.7057, 4.7988, 5.0608,
         5.0076]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[3., 4., 2., 1., 5., 0., 6., 7., 9., 8.]], device='cuda:0')
tensor([[2., 8., 3., 1., 7., 5., 9., 0., 4., 6.]], device='cuda:0')
8.349573771158854
tensor([[3.4836, 3.5236, 3.7938, 3.9644, 4.3346, 4.7014, 4.8977, 5.0099, 5.6961,
         6.2525]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]], device='cuda:0')
tensor([[4., 3., 1., 7., 0., 5., 9., 2., 8., 6.]], device='cuda:0')
8.33335018157959
tensor([[5.2327, 5.1555, 4.7622, 4.7282, 4.6454, 4.4322, 4.6331, 4.3813, 3.9482,
         4.

8.330539067586264
tensor([[5.2881, 5.1284, 4.6839, 4.5120, 4.4275, 4.4154, 4.4009, 4.5729, 4.7968,
         4.7379]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[6., 5., 4., 3., 7., 2., 9., 8., 1., 0.]], device='cuda:0')
tensor([[5., 2., 6., 4., 3., 8., 7., 9., 0., 1.]], device='cuda:0')
8.330358505249023
tensor([[5.4680, 5.5788, 4.8481, 4.6516, 4.6167, 4.3454, 4.0640, 4.0141, 3.3427,
         3.2230]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[9., 8., 7., 6., 5., 4., 3., 2., 0., 1.]], device='cuda:0')
tensor([[7., 6., 4., 3., 8., 5., 9., 1., 0., 2.]], device='cuda:0')
8.338736216227213
tensor([[4.5644, 4.7132, 4.8832, 4.6534, 4.6586, 4.2033, 4.4797, 4.1260, 4.8892,
         5.6394]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[7., 5., 6., 0., 3., 4., 1., 2., 8., 9.]], device='cuda:0')
tensor([[1., 7., 2., 8., 9., 6., 5., 3., 0., 4.]], device='cuda:0')
8.342211723327637
tensor([[2.5064, 2.9618, 4.2011, 4.6640, 4.5248, 4.7606, 4.5982, 4.9576, 5.4082,
         5

8.327406565348307
tensor([[5.7337, 5.5025, 4.9766, 4.5552, 4.6426, 4.0890, 4.8845, 3.8934, 3.5892,
         4.1976]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[8., 7., 5., 9., 3., 4., 6., 2., 1., 0.]], device='cuda:0')
tensor([[8., 4., 1., 9., 2., 7., 5., 3., 0., 6.]], device='cuda:0')
8.33234437306722
tensor([[3.1003, 3.1878, 3.9174, 4.3954, 4.4167, 4.7071, 5.0111, 4.8380, 4.9332,
         5.3388]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[0., 1., 2., 3., 4., 5., 7., 8., 6., 9.]], device='cuda:0')
tensor([[4., 1., 9., 0., 8., 2., 5., 3., 6., 7.]], device='cuda:0')
8.335553487141928
tensor([[2.6702, 3.0733, 3.7531, 4.2490, 4.1519, 4.8384, 4.4650, 5.1435, 5.9332,
         6.2753]], device='cuda:0', grad_fn=<SelectBackward>)
tensor([[0., 1., 2., 4., 3., 6., 5., 7., 8., 9.]], device='cuda:0')
tensor([[6., 0., 4., 2., 7., 1., 5., 3., 8., 9.]], device='cuda:0')
8.331332524617514
tensor([[4.3853, 3.9912, 3.9784, 4.4832, 4.3260, 4.6305, 4.7713, 4.7784, 4.7690,
         4.

In [ ]:
model.requires_grad

In [ ]:
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(input1, input2)
output.shape

In [ ]:
(before_param_data[name] == param.data).any()

In [ ]:
param.data

In [ ]:
assert False, "AA"

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)